In [192]:
import pandas as pd
import glob
import re
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import hex_to_rgb
from nhl_themes import teamColours
pd.set_option('mode.chained_assignment', None)

In [193]:
data = pd.read_csv("EH_std_gl_stats_all_regular_adj_2024-02-08.csv")
data = data.loc[data["xGA"] > data["xGA"].quantile(0.25)]
data["Player"] = data["Player"].str.split(n=1).str[1]
data = data.set_index(["Player"])

In [194]:
data["GSAx/60"] = round((60 * data["GSAx"]) / data["TOI"], 2)
data.head()

,Season,Team,Position,GP,TOI,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAA,GSAx,GSAx/60
Player,,,,,,,,,,,,,,,,
Hill,23-24,VGK,G,18,1021.13,32.17,505.96,766.12,52.10,93.64,95.80,93.20,2.60,16.49,19.93,1.17
Schmid,23-24,N.J,G,15,755.53,41.62,386.96,536.85,38.61,89.24,92.25,92.81,-0.56,-4.40,-3.01,-0.24
Lyon,23-24,DET,G,21,1192.97,50.03,638.11,902.90,60.62,92.16,94.46,93.29,1.17,11.33,10.59,0.53
Nedeljkovic,23-24,PIT,G,16,898.98,38.96,469.03,685.64,43.44,91.69,94.32,93.66,0.65,6.15,4.48,0.30
Georgiev,23-24,COL,G,42,2456.87,117.61,1152.36,1628.15,119.14,89.79,92.78,92.68,0.09,-6.79,1.53,0.04


In [195]:
data["GA/100 xGA"] = round((100 * data["GA"] / data["xGA"]))
data.head()

,Season,Team,Position,GP,TOI,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAA,GSAx,GSAx/60,GA/100 xGA
Player,,,,,,,,,,,,,,,,,
Hill,23-24,VGK,G,18,1021.13,32.17,505.96,766.12,52.10,93.64,95.80,93.20,2.60,16.49,19.93,1.17,62.0
Schmid,23-24,N.J,G,15,755.53,41.62,386.96,536.85,38.61,89.24,92.25,92.81,-0.56,-4.40,-3.01,-0.24,108.0
Lyon,23-24,DET,G,21,1192.97,50.03,638.11,902.90,60.62,92.16,94.46,93.29,1.17,11.33,10.59,0.53,83.0
Nedeljkovic,23-24,PIT,G,16,898.98,38.96,469.03,685.64,43.44,91.69,94.32,93.66,0.65,6.15,4.48,0.30,90.0
Georgiev,23-24,COL,G,42,2456.87,117.61,1152.36,1628.15,119.14,89.79,92.78,92.68,0.09,-6.79,1.53,0.04,99.0


In [197]:
fig = go.Figure()
for row in data.index:
    print(teamColours[data.loc[row, 'Team']])
    # fig.add_annotation(x=data.loc[row, 'xGA'], y=data.loc[row, 'GA/100 xGA'], 
    #         text='Player', showarrow=False, 
    #         font=dict(size=11, color=teamColours[data.loc[row, 'Team']['text']]),
    #         bgcolor=teamColours[data.loc[row, 'Team']['colour']], opacity=0.85, 
    #         borderpad=2.5)
    
fig.update_layout(showlegend=False, width=750, height=750)
fig.update_xaxes(range=[(data["xGA"].min() - 5), (data["xGA"].max() + 5)])
fig.update_yaxes(autorange="reversed")
fig.show()

{'colour': '#B4975A', 'text': '#000000'}


KeyError: 'N.J'